# Statistical birds eye view of the contents in an AiiDAdb

This is the first of two deliverable for the SiSc-Lab2020 project.

Authors = Miao Wang(a - e), Zhipeng Tan(f - i)

Supervisors: Jens Bröder, Dr. Daniel Wortmann, Johannes Wasmer, Prof. Dr. Stefan Blügel.

In [ ]:
# Instructions by supervisors

## Jens
a= """
You have to implement this notebook.

In the end only text (markdown) cells and output results of code cells should be seen if one hides the code cells (hide_code extension).

That can easily exported into a nice pdf file (google it, probably will find sth with `nbconvert`).

Also the amount of python code in this notebook should be minimal.

Rather, export the functions you use to python file(s) and import them here (hide complexity).

Optional dump query results in a file, from which results will be reread for speed, i.e cache results.
"""

## Johannes
a = '''
After talking with Jens about it, here are some additional thoughts on the code structure and implementation, for both deliverables.

The **primary** goal is of course that the code should work, produce nice output (and helpful error messages), obviously.

The **secondary** goal is speed. How long do you expect your code to run on a dataset of a given size? Are there multiple paths to a goal, but with differing performance?

You can break the runtime down into several steps: data acquisition, data transformation (or preprocessing), data analysis, data visualization. In this project, we will rename/replace these steps to: **querying, de-/serialization, analysis=visualization**.

**Querying the database.** Performance considerations:
- Performance measurement: use the magics `%time` and `%timeit`.
- Query evaluations: queries (in general) use 'lazy evaluation'.
  - *Query building* methods build the query but do not execute it. These are chainable methods like `append()`, `get_outgoing()`, etc.
  - *Query execution* methods send the query to the database to be evaluated. There are two kinds:
    - non-iterator methods: e.g. `all()`, `first()`, etc. These return a result `list`: all items are loaded into memory.
    - iterator methods: e.g. `iterall()`, `iterdict()`. These return a result `Generator`: only one item at a time is loaded into memory.
    
**De-/serialization**, i.e. writing and reading it to/from a file. *Keep in mind: if you come to the conclusion this is unnecessary, then justify it!* Considerations:
- Necessity: we assume 'yes'. So you need serialization/deserialization routine(s).
- Code design: we recommend to write a serializer that moves *all* data needed from aiida to file (perform query & serialization). Then the visualization methods are decoupled from aiida and load data from that file. Advantages: a) only needs to be called when data in database changed, b) similar queries for different visualizations can be performed only once. One design option is this:
  ```python
  serialize = sisclab.Serializer(profile)
  serialize.to_file(filepath)
  visualize = sisclab.Visualizer(filepath)
  visualize.histogram(cumulative=True, plot_options)
  # plots histogram
  ```
- Serialization format: there are two practical options (maybe more):
  - `dict`: tree-like. JSON format. One `dict` per file. choose key-value (nested?) based on use-case. in general, `uuid` is a good key.
  - `pandas.Dataframe`: could be preferrable in some cases.
- Serialization location:
  - one file or several files?
  - we recommend to de/serialize from/to `sisclab/data/` folder. It is included in the project's `.gitignore` file, so nothing in it gets committed to/from git (git is for code, not for data; the code generates the data).
- Transformation:
  - if needed, decide where to put needed data transformations (before serialization or after deserialization) to minimize them.
- Deserialization: 
  - a class (as above) might help to define the deserialization format only once for all visualization methods.
  

**Visualization**:
- Prefer `bokeh` to `matplotlib` or other libs wherever possible, unless you have a good justification.
- In `D1`, static plots are okay, interactive plots are a bonus.
- Lists results (when plot is overkill) will look nicer in a notebook if they are a `pandas.Series` or `pandas.Dataframes`.
- Think about function signatures. Can you generalize them to make a nice interface? For example, a signature for SubtaskD1.c might look like this:
  ```python
  def node_type_summary(user_list : list = [], node_basetype : Node = Data,
                        chart_type : bokeh.chart_type = bokeh.pie_chart, plot : bool = True):
    """
    :param user_list: list of users. empty list = all users = default.
    :param node_basetype: subdivides chart into subtypes. Valid base type examples: ProcessNode, CalculationNode, WorkflowNode, Data, ArrayData.
    :param chart_type: bokeh visualization type. pie chart = default.
    :param plot: True: show plot, don't return data. False: don't plot, return data.
    :return: stats: a dictionary {node_subtype : node_count}, insertion-order sorted in descending order.
    :rtype: dict.
    """
  ```
'''

In [ ]:
# Imports

In [ ]:
# magics:
# # autoreload imports. 
# # intent: if i change sth in import, i don't have to restart kernel. enable only for development.
%load_ext autoreload
%autoreload 2
# # choose matplotlib backend. backend 'notebook' allows interactive plots if your env allows it.
%matplotlib notebook


In [ ]:
# python imports:
import time
import json
#from pprint import pprint

#%pylab inline
#figuresize=(18, 4)
from collections import Counter
from math import pi
import numpy as np
import pandas as pd
from pandas import DataFrame
from bokeh.io import output_file,output_notebook, show
from bokeh.layouts import column
from bokeh.palettes import Category20,Category20c,Spectral11
from bokeh.plotting import figure,ColumnDataSource
from bokeh.transform import cumsum
from bokeh.models import Legend,LegendItem,HoverTool,ColumnDataSource

# aiida imports:
from aiida import load_profile
profile = load_profile()

# ggf add futher imports
from aiida.orm import QueryBuilder as QB
from aiida.orm import QueryBuilder
from aiida.orm import WorkflowNode
from aiida.orm import load_node, Node, Group, Computer,Dict
from aiida.orm import User, CalcJobNode, Code, StructureData, ProcessNode
from aiida.plugins import DataFactory
from aiida.common.constants import elements as PeriodicTableElements

# project imports:
#import helpers
# if this does not work, do a `pip install -e .` in the aiida-jutools head folder
from aiida_jutools.sisc_lab import helpers


import aiida_jutools.sisc_lab.util.data_visu as DV
import aiida_jutools.sisc_lab.util.serialization as SR
from aiida_jutools.sisc_lab.util.data_visu import AnalyseStructureElements,ShowElements


In [ ]:
output_notebook()

In [ ]:
# (example:)
helpers.print_bold(f"This notebook/dashboard will visualize the contents from the database of profile {profile.name}")

# Database overview:

In [ ]:
# SubtaskD1.a: Node information
#Task:

In [ ]:
# query for all nodes
print('Information on nodes in the DB: \n')
now = time.strftime("%c")
print('last executed on {}'.format(now))
q = QB()
q.append(Node, project=['id', 'ctime', 'mtime', 'node_type'], tag='node')
q.append(User, with_node='node', project='email')
# TODO: execute query here
t = time.time()
res = q.all()
elapsed = time.time() - t
totalnodes = len(res)
print("Total number of nodes in the database: {} (retrieved in {} s.)".format(totalnodes, elapsed))

## User information:

In [ ]:
# SubtaskD1.b: Users
a = '''
Task: print out a list of Users and how many nodes belong to them

for example

```
Users:
- j.broeder@fz-juelich.de created 182 nodes
- tests@aiida.mail created 104 nodes
```
'''

In [ ]:
print("Users:")
helpers.print_Count('user',res)

## Node types distribution:

In [ ]:
# SubtaskD1.c: Node types
a = '''
Task: plot node information in two pie chart plots

One showing what data nodes there (with their lowest class names(node_type)) I.e Dict, K-pointsData, CifData, FleurinpData...

And one chart showning the process nodes, (with their lowest class names(process_type) i.e CalcjobNodes: FleurCalcjob, FleurinputgenCalcjob, ...

WorkChain nodes: FleurSCFWorkchain, FleurBandDosWorkchain, ..., calcfunctions, and workfunction nodes are fine to not show the lowest class names
'''

In [ ]:
#node types
print("Node types:")
helpers.print_Count('types',res)

In [ ]:
#split data nodes and process nodes
output_notebook()
types = Counter([r[3] for r in res])
x = helpers.get_data_node_count(types,'data') 
p = helpers.draw_pie_chart(x,'Data Nodes:%s')

x1 = helpers.get_process_node_count(types,'process')
p1 = helpers.draw_pie_chart(x1,'Process Nodes:%s')

show(column(p,p1))


In [ ]:
output_notebook()
p = helpers.draw_pie_chart(Counter(helpers.get_dict_link_types()),'Dict Link Types:%s')
show(p)

## Database time evolution:

In [ ]:
# SubtaskD1.d: Histogram
# Task: Cumulative Histogram/ or line plot by ctime & mtime of all nodes over time

In [ ]:
# line plot by ctime & mtime
users = Counter([r[4] for r in res])
output_notebook()
helpers.draw_line_plot(users,res)

## Codes:

In [ ]:
# SubtaskD1.e: Codes
#Task: List Code names, sorted by by how many calcjobs where run with each

In [ ]:
codes = Code.objects.all()
result = {code.full_label: len(code.get_outgoing(node_class=CalcJobNode).all_nodes()) for code in codes}
#result_df=pd.Series(result).sort_values(ascending=False)
result_df=pd.DataFrame({'code@computer':result.keys(),'CalaJobcount':result.values()}).sort_values(by='CalaJobcount',ascending=False).reset_index(drop=True)
result_df

## Groups:

In [ ]:
# SubtaskD1.f: Groups
#Task: List all group names with how many nodes they contain (verdi group list -C) (exclude import and export groups)

In [ ]:
#load_profile()
#!verdi group list --all
try:
    Groups_data = SR.deserialize_from_file('./output/group.json',Node_type='Group')
except:
    qb = QueryBuilder()
    qb.append(Group)
    group = qb.all()

    #data = GroupDataHelper(group)
    #data.ListGroup(exclude=['export','import'])

    ### add more columns for this and do also for other nodes
    serializer = SR.Serializer(group)
    serializer.to_file('./output/group.json',Node_type='Group')
    Groups_data = SR.deserialize_from_file('./output/group.json',Node_type='Group')

In [ ]:
qb = QueryBuilder()
qb.append(Group)
group = qb.all()
group[0][0].__dict__
s = dir(group[0][0])
s


In [ ]:
group[0][0].count

In [ ]:
Groups_data

In [ ]:
data = DV.GroupDataHelper(x)
data.ListGroup(exclude=['export','import'])

## Structure Analysis:

In [ ]:
# SubtaskD1.g: Structures
a = '''
Task: Further analyze what structures are in the DB

Number of structureData node versus how many atoms they contain. 

here interactive with bokeh hover tool showing the structure formula and uuid

Number of StructureData nodes versus elements bokeh bar chart, since there are over 
100 elements in the periodic table you can split it over several plots, or just use the charge number as in 
'example/element_content.png' but then make it interactive that once one hovers 
with the mouse over a bar it tells you what element it is and how many structures there are containing this element-
'''

In [ ]:
try:
    Newdata = SR.deserialize_from_file(filepath,Node_type = 'StructureFormula')
except:
    ################### serialization
    qb = QueryBuilder()
    qb.append(StructureData)
    StructDatas = qb.all()

    #print(dic.keys())

    serializer = SR.Serializer(StructDatas)
    filepath = './output/Num_structure.json'
    serializer.to_file(filepath ,Node_type='StructureFormula')
    Newdata = SR.deserialize_from_file(filepath,Node_type = 'StructureFormula')


In [ ]:
DV.ShowFormula(Newdata)

In [ ]:
try:
    x = SR.deserialize_from_file(filepath,'StructureElement')
except:
    qb = QueryBuilder()
    qb.append(StructureData)
    StructDatas = qb.all()
    serializer = SR.Serializer(StructDatas)
    filepath = './output/Struct_Element.json'
    serializer.to_file(filepath,'StructureElement')
    x = SR.deserialize_from_file(filepath,'StructureElement')

In [ ]:
ShowElements(x)
## sort in other ways

## Processes:

In [ ]:
# SubtaskD1.h: Calculations
a = '''
Task: more detail analysis of Calculations

`print('\n\nMore detailed analysis of Calculations \n')`

List, stacked Histogram of Calculations types and the state it ended up finished, failed, exit codes, exit messages

more detail analysis of WorkChains

`print('\n\nMore detailed analysis of WorkChains \n')`

List,  stacked Histogram for each Workchain type and the state it ended up in finished, failed, exit codes, exit messages
'''

In [ ]:
####### CalcNode 
try:
    filepath = './output/CalcNode.json'
    calcArray = SR.deserialize_from_file(filepath,Node_type = 'ProcessNode')
except:  
    qb = QueryBuilder()
    qb.append(CalcJobNode)
    CalcNode = qb.all()

    serializer = SR.Serializer(CalcNode)
    filepath = './output/CalcNode.json'
    serializer.to_file(filepath,'ProcessNode')
    calcArray = SR.deserialize_from_file(filepath,Node_type = 'ProcessNode')

######## WorkflowNode
try:
    filepath2 = './output/WorkflowNode.json'
    WorkflowArray = SR.deserialize_from_file(filepath2,Node_type = 'ProcessNode')
except:
    qb = QueryBuilder()
    qb.append(WorkflowNode)
    WorkflowNodes = qb.all()

    serializer = SR.Serializer(WorkflowNodes)
    filepath2 = './output/WorkflowNode.json'
    serializer.to_file(filepath2,'ProcessNode')
    WorkflowArray = SR.deserialize_from_file(filepath2,Node_type = 'ProcessNode')

In [ ]:
calcArray.head()

In [ ]:
WorkflowArray.head()

In [ ]:
Newdict1 = DV.GetWorkflowDict(WorkflowArray)
Newdict2 = DV.GetWorkflowDict(calcArray)
DV.ShowWorkflow(Newdict1,'Work Flow Node Information')
DV.ShowWorkflow(Newdict2,'Calculate Job Node Information')

# Data provenance health indicators:

In [ ]:

# SubtaskD1.i: Provenance
#Task: Database and provenance health: display the number of nodes who have no incomming and outgoing links, no incomming links (any number outgoing), and no outgoing links (any number incomming)

In [ ]:
########## this cell will take some time,but after the preprocessing everything should be fine
try:
    filepath = './output/provenance.json'
    provenance = SR.deserialize_from_file(filepath,'Provenance')
except:
    qb = QueryBuilder()
    qb.append(Node)
    Nodes = qb.all()

    #### serialization to filepath
    provenance_serializer = SR.Serializer(Nodes)
    filepath = './output/provenance.json'
    provenance_serializer.to_file(filepath,'Provenance')
    provenance = SR.deserialize_from_file(filepath,'Provenance')

In [ ]:
#### deserialization from filepath
provenance

In [ ]:

No_Incoming_Mydict,No_Outgoing_Mydict,No_InOut_Mydict = DV.Count_In_Out(provenance)
print(No_Incoming_Mydict,No_Outgoing_Mydict,No_InOut_Mydict)

In [ ]:
DV.Show_In_Out(No_Incoming_Mydict,No_Outgoing_Mydict,No_InOut_Mydict)
### split and think about bar plot
# reduce complexity